# CSE547 - Colab 3
## K-Means & PCA

Adapted From Stanford's CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [13]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 58kB/s 
     |████████████████████████████████| 204kB 1.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=3fe0b8d672e0e0a1b1e495fceedda00fa339f3d77478aac877e48064e82098c5
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we import some of the libraries usually needed by our workload.





In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [16]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-26 04:39:19--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.214.203.143, 34.192.123.246, 54.158.230.58, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.214.203.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  75.3MB/s    in 0.2s    

2020-04-26 04:39:19 (75.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://6f2e5fc3.ngrok.io


### Data Preprocessing

In this Colab, rather than downloading a file from Google Drive, we will load a famous machine learning dataset, the [Breast Cancer Wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), using the ```scikit-learn``` datasets loader.

In [10]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
print(breast_cancer.data)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


For convenience, given that the dataset is small, we first construct a Pandas dataframe, tune the schema, and then convert it into a Spark dataframe.

In [17]:
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df = spark.createDataFrame(pd_df)

def set_df_columns_nullable(spark, df, column_list, nullable=False):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))
print(df)
#df.printSchema()

DataFrame[mean radius: double, mean texture: double, mean perimeter: double, mean area: double, mean smoothness: double, mean compactness: double, mean concavity: double, mean concave points: double, mean symmetry: double, mean fractal dimension: double, radius error: double, texture error: double, perimeter error: double, area error: double, smoothness error: double, compactness error: double, concavity error: double, concave points error: double, symmetry error: double, fractal dimension error: double, worst radius: double, worst texture: double, worst perimeter: double, worst area: double, worst smoothness: double, worst compactness: double, worst concavity: double, worst concave points: double, worst symmetry: double, worst fractal dimension: double, features: array<double>]


With the next cell, we build the two datastructures that we will be using throughout this Colab:


*   ```features```, a dataframe of Dense vectors, containing all the original features in the dataset;
*   ```labels```, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.



In [0]:
from pyspark.ml.linalg import Vectors
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

### Your task

If you run successfully the Setup and Data Preprocessing stages, you are now ready to cluster the data with the [K-means](https://spark.apache.org/docs/latest/ml-clustering.html) algorithm included in MLlib (Spark's Machine Learning library).
Set the ```k``` parameter to **2**, use a seed of **1**, fit the model, and the compute the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) (i.e., a measure of quality of the obtained clustering).  

**IMPORTANT:** use the MLlib implementation of the Silhouette score (via ```ClusteringEvaluator```).

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(features)

# Make predictions
predictions = model.transform(features)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette)+'\n')

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: \n")
for center in centers:
    print(center,'\n')

Silhouette with squared euclidean distance = 0.8342904262826145

Cluster Centers: 

[1.25562991e+01 1.85703653e+01 8.11234703e+01 4.96061872e+02
 9.48844977e-02 9.10998174e-02 6.24377642e-02 3.34325434e-02
 1.78057991e-01 6.34540183e-02 3.04190868e-01 1.21515320e+00
 2.15288059e+00 2.37852922e+01 7.17326256e-03 2.34746895e-02
 2.87455128e-02 1.06363242e-02 2.06135799e-02 3.74750297e-03
 1.40439018e+01 2.47095434e+01 9.19375114e+01 6.19647945e+02
 1.29959110e-01 2.23311758e-01 2.19214947e-01 9.13298425e-02
 2.83553653e-01 8.32819406e-02] 

[1.93799237e+01 2.16945802e+01 1.28231298e+02 1.18592977e+03
 1.01294580e-01 1.48612977e-01 1.76939466e-01 1.00698779e-01
 1.91539695e-01 6.06029008e-02 7.42803817e-01 1.22253817e+00
 5.25058015e+00 9.56781679e+01 6.59868702e-03 3.21766947e-02
 4.24197710e-02 1.56739847e-02 2.03039695e-02 3.95338931e-03
 2.37094656e+01 2.89126718e+01 1.58496183e+02 1.75302290e+03
 1.40424733e-01 3.57757710e-01 4.49306107e-01 1.92431069e-01
 3.11881679e-01 8.61654962e-

Take the predictions produced by K-means, and compare them with the ```labels``` variable (i.e., the ground truth from our dataset).  

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other).

*HINT*: you can use ```np.count_nonzero(series_a == series_b)``` to quickly compute the element-wise comparison of two series.

**IMPORTANT**: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier!  As such, label ```0``` does not necessarily match the cluster identifier ```0```.


In [0]:
predictionCol = predictions.select('prediction').toPandas()
predictionCol2 = predictionCol

c=0
for i in range(len(labels)):
  c+=predictionCol.iloc[i]=labels[i]
c

prediction    486
Name: 0, dtype: int64

Now perform dimensionality reduction on the ```features``` using the [PCA](https://spark.apache.org/docs/latest/ml-features.html#pca) statistical procedure, available as well in MLlib.

Set the ```k``` parameter to **2**, effectively reducing the dataset size of a **15X** factor.

In [0]:
from pyspark.ml.feature import PCA

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(features)

result = model.transform(features).select("pcaFeatures")
result = result.withColumnRenamed('pcaFeatures', 'features')

pcavectors = result.rdd.map(lambda row: Vectors.dense(row.features))
pcafeatures = spark.createDataFrame(pcavectors.map(Row), ["features"])


result.show(truncate=False)
pcafeatures.show()

+-----------------------------------------+
|features                                 |
+-----------------------------------------+
|[-2260.0138862925405,-187.96030122263656]|
|[-2368.993755782054,121.58742425815576]  |
|[-2095.6652015478608,145.11398565870167] |
|[-692.6905100570508,38.57692259208171]   |
|[-2030.2124927427058,295.29798399279264] |
|[-888.280053576076,26.079796157025783]   |
|[-1921.0822124748447,58.80757247309966]  |
|[-1074.781335004796,31.771227808469725]  |
|[-908.5784781618829,63.830752790446354]  |
|[-861.5784494075679,40.57073549705332]   |
|[-1404.559130649947,88.2321825773627]    |
|[-1524.2324408687816,-3.263057316777752] |
|[-1734.385647746415,273.162678151146]    |
|[-1162.9140032230355,217.6348180834463]  |
|[-903.4301030498831,135.61517666084805]  |
|[-1155.8759954206848,76.80889383742193]  |
|[-1335.7294321308066,-2.4684005450354887]|
|[-1547.2640922523087,3.805675972574666]  |
|[-2714.9647651812156,-164.37610864258735]|
|[-908.2502671870874,118.2164200

Now run K-means with the same parameters as above, but on the ```pcaFeatures``` produced by the PCA reduction you just executed. (Again with a seed of **1**)

Compute the Silhouette score, as well as the number of data points that have been clustered correctly.

In [0]:
# Trains a k-means model.
pmodel = kmeans.fit(pcafeatures)

# Make predictions
PCApredictions = pmodel.transform(pcafeatures)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

psilhouette = evaluator.evaluate(PCApredictions)
psilhouette

# Shows the result.
#pcenters = pmodel.clusterCenters()

0.8348610363444837

In [0]:
PCApredictionCol = PCApredictions.select('prediction').toPandas()

c=0
for i in range(len(labels)):
  c+=PCApredictionCol.iloc[i]!=labels[i]
c

prediction    486
Name: 0, dtype: int64

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!